In [17]:
from PIL import Image, ImageChops, ImageEnhance
import numpy as np

def perform_ela(image_path, quality=90):
    # Open the image
    original = Image.open(image_path).convert('RGB')
    # Save it temporarily with the given quality
    compressed_path = "temp.jpg"
    original.save(compressed_path, "JPEG", quality=quality)
    # Open the compressed image
    compressed = Image.open(compressed_path)
    # Compute the difference
    ela_image = ImageChops.difference(original, compressed)
    # Enhance the differences
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return np.array(ela_image)

In [25]:
import os
from tqdm import tqdm

def prepare_dataset(au_dataset, tp_dataset):
    auth_data = []
    auth_labels = []
    
    for image_file in tqdm(os.listdir(au_dataset)):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(au_dataset, image_file)
            ela_image = perform_ela(image_path)
            auth_data.append(ela_image)
            auth_labels.append(0)
    tp_data = []
    tp_labels = []
    for image_file in tqdm(os.listdir(tp_dataset)):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(tp_dataset, image_file)
            ela_image = perform_ela(image_path)
            tp_data.append(ela_image)
            tp_labels.append(1)

    final_data = auth_data + tp_data
    final_labels = auth_labels + tp_labels
    print(len(final_data), len(final_labels))
    
    return np.array(final_data), np.array(final_labels)


In [26]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load the VGG19 model
def create_model(input_shape):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Binary classification
    ])
    
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Assuming data and labels are ready
data, labels = prepare_dataset('CASIA22/Au', 'CASIA22/Tp')


100%|██████████| 5125/5125 [00:19<00:00, 265.99it/s]

9501 9501


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9501,) + inhomogeneous part.

In [ ]:
# Normalize and split the data
data = data / 255.0
labels = to_categorical(labels, num_classes=2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create and train the model
input_shape = (data.shape[1], data.shape[2], data.shape[3])
model = create_model(input_shape)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)